# Features

## 1. long-term position control
## 2. B/S number per trade
## 3. KDJ risk control
## 4. trade log

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from main import Choice

In [2]:

choices = []

# 宽基指数
zz1000 = Choice(p_min=2.073, p_max=3.247, pos_max=20000,
                price=2.708, d_month=59, d_week=63, code="512100.SH")
kc50   = Choice(p_min=0.848, p_max=1.696, pos_max=50000,
                price=1.164, d_month=45, d_week=78, code="588000.SH")
gqhl   = Choice(p_min=1.820, p_max=2.215, pos_max=30000,
                price=1.998, d_month=54, d_week=43, code="501059.SH")
choices.append(zz1000)
choices.append(kc50)
choices.append(gqhl)

# 行业指数
yyetf  = Choice(p_min=0.454, p_max=0.907, pos_max=100000,
                price=0.464, d_month=36, d_week=33, code="512010.SH")
gfetf  = Choice(p_min=1.040, p_max=1.925, pos_max=30000,
                price=1.279, d_month=38, d_week=26, code="515790.SH")
choices.append(yyetf)
choices.append(gfetf)

# 转债银行
jyyh   = Choice(p_min=3.67, p_max=5.38, pos_max=13000,
                price=4.07, d_month=38, d_week=27, code="002807.SZ")
wxyh   = Choice(p_min=4.80, p_max=7.06, pos_max=10000,
                price=5.58, d_month=49, d_week=33, code="600908.SH")
hzyh   = Choice(p_min=8.88, p_max=16.43, pos_max=4000,
                price=11.87, d_month=28, d_week=30, code="600926.SH")
snyh   = Choice(p_min=4.41, p_max=6.77, pos_max=8000,
                price=4.73, d_month=34, d_week=28, code="603323.SH")
choices.append(jyyh)
choices.append(wxyh)
choices.append(hzyh)
choices.append(snyh)

512100.SH: buy_price_position
	price: 3.247, position: 0.000
	price: 2.777, position: 0.100
	price: 2.496, position: 0.300
	price: 2.256, position: 0.600
	price: 2.052, position: 1.000
512100.SH: sell_price_position
	price: 3.334, position: 0.000
	price: 2.928, position: 0.400
	price: 2.590, position: 0.700
	price: 2.308, position: 0.900
	price: 2.073, position: 1.000
588000.SH: buy_price_position
	price: 1.696, position: 0.000
	price: 1.357, position: 0.100
	price: 1.153, position: 0.300
	price: 0.980, position: 0.600
	price: 0.833, position: 1.000
588000.SH: sell_price_position
	price: 1.759, position: 0.000
	price: 1.465, position: 0.400
	price: 1.221, position: 0.700
	price: 1.018, position: 0.900
	price: 0.848, position: 1.000
501059.SH: buy_price_position
	price: 2.215, position: 0.000
	price: 2.057, position: 0.100
	price: 1.962, position: 0.300
	price: 1.882, position: 0.600
	price: 1.813, position: 1.000
501059.SH: sell_price_position
	price: 2.244, position: 0.000
	price: 2.1

## 1. long-term position control

In [3]:
for choice in choices:
    long_pos = choice.cal_position(choice.price, True)
    short_pos = choice.cal_position(choice.price, False)
    print(f"{choice.code}: price: {choice.price:.3f},"
          f"\n\tlong position: {long_pos:.3f}, {long_pos * choice.pos_max:.0f}"
          f"\n\tshort position: {short_pos:.3f}, {short_pos * choice.pos_max:.0f}")


512100.SH: price: 2.708,
	long position: 0.149, 2985
	short position: 0.595, 11898
588000.SH: price: 1.164,
	long position: 0.289, 14473
	short position: 0.756, 37807
501059.SH: price: 1.998,
	long position: 0.224, 6734
	short position: 0.689, 20668
512010.SH: price: 0.464,
	long position: 0.908, 90787
	short position: 0.989, 98896
515790.SH: price: 1.279,
	long position: 0.432, 12968
	short position: 0.842, 25249
002807.SZ: price: 4.070,
	long position: 0.485, 6310
	short position: 0.872, 11333
600908.SH: price: 5.580,
	long position: 0.322, 3219
	short position: 0.779, 7791
600926.SH: price: 11.870,
	long position: 0.270, 1080
	short position: 0.737, 2947
603323.SH: price: 4.730,
	long position: 0.647, 5175
	short position: 0.932, 7458


## 2. buy/sell number per trade

#### 1. standard situation
set 1/40=2.5%

if trade while 3% decrease with 1/40 = 2.5% position, 
from top 100% price to bottom 50% price, we should trade about 23 times(0.97^23=0.51), we get 23*2.5% = 57.5% position

that seems ok because KDJ will hint us and the buy diff will decrease from 3% to 1%.

if trade every 1%, trade will happen about 68 times(0.99^68=0.50)

#### 2. scaled situation
set 1/40*scale=2.5%*scale, scale=ln(0.5)/ln(bottom_price)

from top 100% price to bottom 68% price, 3% diff will trade about 13 times($0.97^{13}=0.67$), we get $13*2.5% = 32.5%$ position; 1% diff will trade about 38 times($0.99^38=0.68$), we get $38*2.5%=95.0%$ position.

price range decreased $64%=(1-0.68)/(1-0.5)$
trade number decreased $43%=(23-13)/23$ and $44%=(68-38)/68$

To get similar position, we should trade more: 
trade number should increased: $77%=(23-13)/13$ and $79%=(68-38)/38$

summerize the theoretical formula:
- $0.97^{times_{standard}} = 0.5$
- $0.97^times_scaled   = bottom_price$
- $scale = times_standard / times_scaled$

so, we get:
- $ln(0.97) * times_standard = ln(0.5)$
- $ln(0.97) * times_scaled   = ln(bottom_price)$
- $scale = ln(0.5) / ln(bottom_price)$

for bottom_price = 0.68, $scale = ln(0.5) / ln(0.68) = 1.80$, consistent with the above calculation 77% and 79%.

In [4]:
for choice in choices:
    num_per_trade = choice.cal_pos_number_per_trade()
    print(f"{choice.code}: number_per_trade: {num_per_trade:.0f}")

512100.SH: number_per_trade: 772
588000.SH: number_per_trade: 1250
501059.SH: number_per_trade: 2647
512010.SH: number_per_trade: 2504
515790.SH: number_per_trade: 844
002807.SZ: number_per_trade: 589
600908.SH: number_per_trade: 449
600926.SH: number_per_trade: 113
603323.SH: number_per_trade: 323


## 3. KDJ risk control

In [5]:
for choice in choices:
    risk_density = choice.cal_risk_and_grid_density(choice.d_month, choice.d_week)
    print(f"{choice.code}: price: {choice.price:.3f},\n\t"
          f"risk level: {risk_density[0]}\n\t"
          f"buy sell contrast and diff: {risk_density[1:]}")

512100.SH: price: 2.708,
	risk level: 风险平衡
	buy sell contrast and diff: (0.03, 0.03, 0.03, 0.03)
588000.SH: price: 1.164,
	risk level: 很大风险
	buy sell contrast and diff: (0.04, 0.03, 0.03, 0.025)
501059.SH: price: 1.998,
	risk level: 风险平衡
	buy sell contrast and diff: (0.03, 0.03, 0.03, 0.03)
512010.SH: price: 0.464,
	risk level: 风险平衡
	buy sell contrast and diff: (0.03, 0.03, 0.03, 0.03)
515790.SH: price: 1.279,
	risk level: 风险平衡
	buy sell contrast and diff: (0.03, 0.03, 0.03, 0.03)
002807.SZ: price: 4.070,
	risk level: 风险平衡
	buy sell contrast and diff: (0.03, 0.03, 0.03, 0.03)
600908.SH: price: 5.580,
	risk level: 风险平衡
	buy sell contrast and diff: (0.03, 0.03, 0.03, 0.03)
600926.SH: price: 11.870,
	risk level: 很大机会
	buy sell contrast and diff: (0.03, 0.02, 0.04, 0.03)
603323.SH: price: 4.730,
	risk level: 风险平衡
	buy sell contrast and diff: (0.03, 0.03, 0.03, 0.03)


## 4. trade log

- add a trade transaction
- load from csv log
- save to csv log
